### Imports

In [15]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence

# %run train.ipynb
%run train_arabic.ipynb

[['ق', 'و', 'ل', 'ه'], ['ل', 'ع', 'د', 'م'], ['م', 'ا'], ['ت', 'ت', 'ع', 'ل', 'ق'], ['إ', 'ل', 'خ'], ['أ', 'ي'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['ق', 'و', 'ل', 'ه'], ['م', 'ا'], ['م', 'ر'], ['أ', 'ي'], ['ق', 'ب', 'ي', 'ل'], ['ق', 'و', 'ل'], ['ا', 'ل', 'م', 'ت', 'ن'], ['ل', 'غ', 'ت'], ['و', 'ل', 'و'], ['ا', 'ق', 'ت', 'ص', 'ر'], ['ع', 'ل', 'ى'], ['أ', 'و', 'ص', 'ي', 'ت'], ['ل', 'ه'], ['ب', 'ش', 'ا', 'ة'], ['أ', 'و'], ['أ', 'ع', 'ط', 'و', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ل', 'ا'], ['غ', 'ن', 'م'], ['ل', 'ه'], ['ع', 'ن', 'د'], ['ا', 'ل', 'م', 'و', 'ت'], ['ه', 'ل'], ['ت', 'ب', 'ط', 'ل'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['أ', 'و'], ['ي', 'ش', 'ت', 'ر', 'ى'], ['ل', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ي', 'ؤ', 'خ', 'ذ'], ['م', 'ن'], ['ق', 'و', 'ل', 'ه'], ['ا', 'ل', 'آ', 'ت', 'ي'], ['ك', 'م', 'ا'], ['ل', 'و'], ['ل', 'م'], ['ي', 'ق', 'ل'], ['م', 'ن'], ['م', 'ا', 'ل', 'ي'], ['و', 'ل', 'ا'], ['م', 'ن'], ['غ', 'ن', 'م', 'ي'], ['أ', 'ن', 'ه', 'ا'], ['ل', 'ا'], ['ت', 'ب', 'ط', 'ل'], ['و', 'ع', 'ب', 'ا', 'ر',

### Validation

In [16]:
def validate(model, val_dataset, val_labels, batch_size=BATCH_SIZE):
    """
    This function implements the validation logic
    Inputs:
    - model: the trained model
    - val_dataset: the validation set
    - batch_size: integer representing the number of examples per step
    """

    # (1) create the dataloader for the validation set (make shuffle=False)
    tensor_val_dataset = TensorDataset(val_dataset, val_labels)
    val_dataloader = DataLoader(tensor_val_dataset, batch_size=batch_size, shuffle=False)

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    total_acc_val = 0

    with torch.no_grad():
        for val_input, val_label in tqdm(val_dataloader):

            # Move the validation input to the device
            val_label = val_label.to(device)

            # Move the validation label to the device
            val_input = val_input.to(device)

            # Do the forward pass
            output = model(val_input).float()

            # Calculate the batch accuracy
            correct_predictions = (output.argmax(dim=2) == val_label)
            acc = correct_predictions.sum().item()
            total_acc_val += acc

    # Calculate metrics for the entire validation set
    val_accuracy = total_acc_val / (len(val_dataset) * len(val_dataset[0]))

    print(f'Validation Accuracy: {val_accuracy} | DER: {1 - val_accuracy}\n')


### Prepare the data

In [17]:
valid_corpus = readFile(VAL_PATH)

X_val = []
y_val = []

for sentence in valid_corpus:
	# Clean each sentence in the corpus
	# Get the char list for each word in the sentence and its corresponding diacritics
	char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

	X_val.append(char_list)
	y_val.append(diacritics_list)

X_val_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in X_val for word in sentence ]
X_val_padded = pad_sequence(X_val_padded, batch_first=True)

y_val_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_val for word in sentence ]
# print(y_val_padded)
y_val_padded = pad_sequence(y_val_padded, batch_first=True)
# print(y_val_padded)

### Load the model

In [18]:
# Load the saved RNN model for inference
loaded_CRF_model = RNN_CRF(VOCAB_SIZE, LABELS_SIZE)
loaded_CRF_model.load_state_dict(torch.load("./models/crf.pth", map_location=torch.device('cpu')))
loaded_CRF_model.eval()
validate(loaded_CRF_model, X_val_padded, y_val_padded)

  0%|          | 0/415 [00:00<?, ?it/s]

100%|██████████| 415/415 [01:04<00:00,  6.46it/s]

Validation Accuracy: 0.9567667738968618 | DER: 0.04323322610313818

